In [1]:
# Import library-library
import os
import glob
from tqdm import tqdm
import numpy as np

# Data Preparation and Preprocessing
import pandas as pd
import re
from string import digits

# Word Embedding
import joblib
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

# Input and Expansion Query
import nltk
nltk.download('punkt')
import math
from textblob import TextBlob
from yake import KeywordExtractor
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk import tokenize
from operator import itemgetter
from nltk.tokenize import word_tokenize
from nltk.tag import CRFTagger
from textblob import TextBlob
NLTK_StopWords = stopwords.words('indonesian')

from sklearn.tree import DecisionTreeClassifier
import random

from spacy import load, displacy
import time
import datetime
from datetime import datetime as dt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
excluded_words = ["tempat", "waktu", "hari"]

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
# txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
# NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [3]:
def preprocessingCorpus(berita):
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

ct =CRFTagger()
ct.set_model_file('model/TagIndo/all_indo_man_tag_corpus_model.crf.tagger')
def preprocessingKeparahan(berita):
    s = berita
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub("\(.*?\)", "", s)
    # s = s.lower()
    tokens = word_tokenize(s)
    return tokens

def preprocessingNER(berita):
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    # tokens = [token for token in s.split(" ") if token != ""]
    return s

In [4]:
df_train = pd.read_csv('dataset/df_train.csv')
df_train = df_train[pd.notnull(df_train['description'])]
print(df_train.info())
print ('-'*90)
# document_text_train= joblib.load('document_text_train.pkl')
print(len(df_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3686 entries, 0 to 3685
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        3685 non-null   object
 1   date         3686 non-null   object
 2   description  3686 non-null   object
 3   source       3686 non-null   object
dtypes: object(4)
memory usage: 144.0+ KB
None
------------------------------------------------------------------------------------------
3686


In [30]:
df_test = pd.read_csv('dataset/df_test_noPre2.csv')
df_test = df_test[pd.notnull(df_test['description'])]
print(df_test.info())
print ('-'*90)
# document_text_train= joblib.load('document_text_train.pkl')
print(len(df_test))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        100 non-null    object
 1   date         100 non-null    object
 2   description  100 non-null    object
 3   source       100 non-null    object
dtypes: object(4)
memory usage: 3.9+ KB
None
------------------------------------------------------------------------------------------
100


# What

In [6]:
def cari_dokpertama(kueriAsli):
    kueriPre=preprocessingCorpus(kueriAsli)
    kueriPre= " ".join (kueriPre)
    hasilSearch=[]
    tfidf_train = joblib.load('model/QE/tfidf_train.pkl')
    tfidf_vectorizer = joblib.load('model/QE/vectorizer.pkl')
    
    query_vec= tfidf_vectorizer.transform([kueriPre])

    results=cosine_similarity(tfidf_train, query_vec).reshape((-1))
    for i in results.argsort()[-5:][::-1]:
        hasilSearch.append(df_train.iloc[i,-2])
    hasilSearch=". ".join(hasilSearch)
    return hasilSearch



In [7]:
##Keywords Extraction with YAKE
def keyword_yake(hasilSearch):
    keywordYake=[]

    k_extractor = KeywordExtractor(lan="id", n=1, top=50)
    k_extractor2 = KeywordExtractor(lan="id", n=2, top=50)

    keywords = k_extractor.extract_keywords(text=hasilSearch)
    keywords = k_extractor2.extract_keywords(text=hasilSearch)

    keywordYake = [x for x, y in keywords]
    return keywordYake

In [8]:
#Keywords Extraction with TFIDF
def keyword_tfidf(hasilSearch):

    keywordtfidf=[]
    keywordtfidf2=[]

    #doc = 'بَاب فرض الْوضُوء وسننه وهيآته وَفرض الْوضُوء سِتّ خِصَال النِّيَّة عمند غسل الْوَجْه وَغسل الْوَجْه وَغسل الذراعين مَعَ الْمرْفقين وَمسح مَا قل من الرَّأْس وَغسل الرجلَيْن مَعَ الْكَعْبَيْنِ وَالتَّرْتِيب وعَلى قَول الْوَلَاء وسننه عشر خِصَال خمس مِنْهَا قبل غسل الْوَجْه وَهِي التَّسْمِيَة وَغسل الْكَفَّيْنِ والمضمضة وَالِاسْتِنْشَاق وَالْمُبَالغَة فيههما إِلَّا للصَّائِم وَخمْس بعد غسل الْوَجْه وَهِي تَقْدِيم الْيُمْنَى على ليسرى وَمسح جَمِيع الرَّأْس وَمسح الْأُذُنَيْنِ ظاهرهما وباطنهما وَإِدْخَال الأصبعين فيهمَا وتخليل أَصَابِع الرجلَيْن . وَغسل دَاخل الْكَعْبَيْنِ وَلَيْسَ مسح لعنق من سنَنه وفضيلته تكراره ثَلَاثًا وزالواجب فِيهِ مرّة والمرتان أفضل وَالثَّلَاث أكمل وهيآته أَن يبْدَأ فِي تَطْهِير الْأَعْضَاء بمواضع الِابْتِدَاء . فَإِن اقْتصر على فروضه استة أَجزَأَهُ وَإِن ضيع حَظّ نَفسه فِيمَا ترك'
    total_words = re.sub(r'[^\w]', ' ', hasilSearch)
    total_words = total_words.lower().split()
    total_word_length = len(total_words)
    total_sentences = tokenize.sent_tokenize(hasilSearch)
    total_sent_len = len(total_sentences)

    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))

    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}

    def get_top_n(dict_elem, n):
        result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
        hasil =list(result.keys()) 
        return hasil
    keywordtfidf.append(get_top_n(tf_idf_score, 35))
    for i in range(len(keywordtfidf)):
        totalKw=0
        totalKw=len(keywordtfidf[i])
        for j in range(totalKw):
            keywordtfidf2.append(keywordtfidf[i][j])
    return keywordtfidf2

In [9]:
#Keywords Extraction with BERT
def keyword_bert(hasilSearch):

    keywordbert=[]

    keyword1 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 1))
    keyword2 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 2))

    for i in range (0,len (keyword1)):
        keywordbert.append(keyword1[i][0])
        keywordbert.append(keyword2[i][0])
    return keywordbert

In [10]:
def rangking (keywordGabung):
    keywordTemp=[]
    keywordFinal=[]

    def borda_sort(lists):
        scores = {}
        for l in lists:
            for idx, elem in enumerate(reversed(l)):
                if not elem in scores:
                    scores[elem] = 0
                scores[elem] += idx
        return sorted(scores.keys(), key=lambda elem: scores[elem], reverse=True)

    keywordTemp.append(borda_sort(keywordGabung))
    # print ('kandidat temp',keywordTemp)
    # print ('Total Kandidat temp: ',len(keywordTemp[0]))

    if len(keywordTemp[0])>30:
        # print ('kurang dari 80')
        for i in range (0,80):
            keywordFinal.append(keywordTemp[0][i])
    elif len(keywordTemp[0])<80:
        # print ('lebih dari 80')
        for i in range (0,len(keywordTemp)):
            for j in range (0,len(keywordTemp[0])):
                keywordFinal.append(keywordTemp[0][j])
    # print ('Total Kandidat final: ',len(keywordFinal))
    # print ('Kandidat final: ',keywordFinal)
    return keywordFinal


In [11]:
def keyword_BOW(keywordBOW, kueriAsli):
    cekDuplicate = []
    kandidatFix = []

    for i in keywordBOW:
        if(i not in cekDuplicate and i!=0):
            cekDuplicate.append(i)

    queries=kueriAsli
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(cekDuplicate)
    
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 2200
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFix.append(cekDuplicate[idx])
    return kandidatFix

In [12]:
def kandidatFix(kueriAsli,bow):
    #kueri what
    # kueriAsli='apa sebenarnya kejadian yang terjadi diberita tersebut'
    kueri=preprocessingCorpus(kueriAsli)
    kueri= [" ".join (kueri)]
    # print (kueri)
    hasilkandidat=[]
    keywordGabung=[]
    kandidatFix=[]
    kueriFix=[]
    kueriFixWithDelimiter=[]
    


    hasilSearch=cari_dokpertama(kueriAsli)
    keywordYake=keyword_yake(hasilSearch)
    keywordtfidf2=keyword_tfidf(hasilSearch)
    keywordbert=keyword_bert (hasilSearch)
    keywordBOW=keyword_BOW(bow, kueri)

    keywordGabung.append(keywordYake)
    keywordGabung.append(keywordtfidf2)
    keywordGabung.append(keywordbert)
    # keywordGabung.append(keywordBOW)
    hasilrank=rangking(keywordGabung)

    for i in hasilrank:
        kueriFix.append(i)
    for x in keywordBOW:
        kueriFix.append(x)
    for j in kueriFix:
        hasilkandidat.append(j)
    kueriFixWithDelimiter=kueriFix
    kueriFix=[preprocessingCorpus(i) for i in kueriFix]
    for i in kueriFix:
        for j in i:
            kandidatFix.append(j)
    kandidatFix= [" ".join (kandidatFix)]
    
    print ('*'*120)
    return(kandidatFix,keywordGabung,keywordBOW,hasilrank,kueriFixWithDelimiter)


In [13]:
kueriAsliWhat='apa sebenarnya kejadian kecelakaan yang terjadi diberita tersebut'

bow_kecelakaan_text = joblib.load('model/QE/bow_kecelakaan_text.pkl')

kandidatFixWhat,keywordGabungWhat,keywordBOW_What, hasilrankWhat, kueriFixWithDelimiter_What = kandidatFix(kueriAsliWhat, bow_kecelakaan_text)



************************************************************************************************************************


# Keparahan

In [14]:
def structureCheck(indeks,arr):
    if(arr[0][indeks][1]=='VB' or arr[0][indeks][1]=='IN' or arr[0][indeks][1]=='NN' or arr[0][indeks][1]=='NNP' or arr[0][indeks][1]=='SC' or arr[0][indeks][1]=='JJ' or arr[0][indeks][1]=='PR'  or arr[0][indeks][1]=='NND'  or arr[0][indeks][1]=='RB'  or arr[0][indeks][1]=='SYM'  or arr[0][indeks][1]=='RP' or arr[0][indeks][1]=='PRP' or arr[0][indeks][1]=='DT' or arr[0][indeks][1]=='FW' or arr[0][indeks][1]=='MD' or arr[0][indeks][1]=='NEG' or arr[0][indeks][1]=='UH' or arr[0][indeks][1]=='VH'):
        return 1
    else:
        return 0
def cdStructureCheck(indeks,arr):
    if(arr[0][indeks][1]=='CD' or arr[0][indeks][1]=='OD' or arr[0][indeks][0]=='Sang' or arr[0][indeks][0]=='sang' or arr[0][indeks][0]=='Seorang'  or arr[0][indeks][0]=='seorang'  or arr[0][indeks][0]=='seseorang' or arr[0][indeks][0]=='Seseorang' or arr[0][indeks][0]=='Empat' or arr[0][indeks][0]=='Lima' or arr[0][indeks][0]=='Enam' or arr[0][indeks][0]=='Tujuh' or arr[0][indeks][0]=='Delapan' or arr[0][indeks][0]=='Sembilan' or arr[0][indeks][0]=='Sepuluh'  or arr[0][indeks][0]=='Pasangan'  or arr[0][indeks][0]=='pasangan'  or arr[0][indeks][0]=='belasan'  or arr[0][indeks][0]=='Belasan'):
        return 1
    else:
        return 0

In [15]:
def struktur1(arr,indeks):
    
    if(indeks-2>=0 
        and cdStructureCheck(indeks-1,arr) 
        and cdStructureCheck(indeks-2,arr)):
        return (arr[0][indeks-2][0] + ' ' + arr[0][indeks-1][0])
    elif(indeks+2<=len(arr[0])-1 
        and cdStructureCheck(indeks+1,arr) 
        and cdStructureCheck(indeks+2,arr)):
        return (arr[0][indeks+1][0] + ' ' + arr[0][indeks+2][0])
    elif(indeks-1>=0 
        and cdStructureCheck(indeks-1,arr)):
        return arr[0][indeks-1][0]
    elif(indeks+1<=len(arr[0])-1 
        and cdStructureCheck(indeks+1,arr)):
        return arr[0][indeks+1][0]
    elif(indeks-3>=0 
        and structureCheck(indeks-1,arr) 
        and cdStructureCheck(indeks-2,arr) and cdStructureCheck(indeks-3,arr)):
        return (arr[0][indeks-3][0] + ' ' + arr[0][indeks-2][0])
    elif(indeks+3<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) 
        and cdStructureCheck(indeks+2,arr) and cdStructureCheck(indeks+3,arr)):
        return (arr[0][indeks+2][0] + ' ' + arr[0][indeks+3][0])
    elif(indeks-2>=0 
        and structureCheck(indeks-1,arr) 
        and cdStructureCheck(indeks-2,arr)):
        return arr[0][indeks-2][0]
    elif(indeks+2<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) 
        and cdStructureCheck(indeks+2,arr)):
        return arr[0][indeks+2][0]
    elif(indeks-4>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) 
        and cdStructureCheck(indeks-3,arr) and cdStructureCheck(indeks-4,arr)):
        return (arr[0][indeks-4][0] + ' ' + arr[0][indeks-3][0])
    elif(indeks+4<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) 
        and cdStructureCheck(indeks+3,arr) and cdStructureCheck(indeks+4,arr)):
        return (arr[0][indeks+3][0] + ' ' + arr[0][indeks+4][0])
    elif(indeks-3>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) 
        and cdStructureCheck(indeks-3,arr)):
        return arr[0][indeks-3][0]
    elif(indeks+3<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) 
        and cdStructureCheck(indeks+3,arr)):
        return arr[0][indeks+3][0]
    elif(indeks-5>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) 
        and cdStructureCheck(indeks-4,arr) and cdStructureCheck(indeks-5,arr)):
        return (arr[0][indeks-5][0] + ' ' + arr[0][indeks-4][0])
    elif(indeks+5<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) 
        and cdStructureCheck(indeks+4,arr) and cdStructureCheck(indeks+5,arr)):
        return (arr[0][indeks+4][0] + ' ' + arr[0][indeks+5][0])
    elif(indeks-4>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) 
        and cdStructureCheck(indeks-4,arr)):
        return arr[0][indeks-4][0]
    elif(indeks+4<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) 
        and cdStructureCheck(indeks+4,arr)):
        return arr[0][indeks+4][0]
    elif(indeks-6>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and cdStructureCheck(indeks-5,arr) and cdStructureCheck(indeks-6,arr)):
        return (arr[0][indeks-6][0] + ' ' + arr[0][indeks-5][0])
    elif(indeks+6<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and cdStructureCheck(indeks+5,arr) and cdStructureCheck(indeks+6,arr)):
        return (arr[0][indeks+5][0] + ' ' + arr[0][indeks+6][0])
    elif(indeks-5>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and cdStructureCheck(indeks-5,arr)):
        return arr[0][indeks-5][0]
    elif(indeks+5<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and cdStructureCheck(indeks+5,arr)):
        return arr[0][indeks+5][0]
    elif(indeks-7>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) 
        and cdStructureCheck(indeks-6,arr) and cdStructureCheck(indeks-7,arr)):
        return (arr[0][indeks-7][0] + ' ' + arr[0][indeks-6][0])
    elif(indeks+7<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) 
        and cdStructureCheck(indeks+6,arr) and cdStructureCheck(indeks+7,arr)):
        return (arr[0][indeks+6][0] + ' ' + arr[0][indeks+7][0])
    elif(indeks-6>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) 
        and cdStructureCheck(indeks-6,arr)):
        return arr[0][indeks-6][0]
    elif(indeks+6<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) 
        and cdStructureCheck(indeks+6,arr)):
        return arr[0][indeks+6][0]
    elif(indeks-8>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) 
        and cdStructureCheck(indeks-7,arr) and cdStructureCheck(indeks-8,arr)):
        return (arr[0][indeks-8][0] + ' ' + arr[0][indeks-7][0])
    elif(indeks+8<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) 
        and cdStructureCheck(indeks+7,arr) and cdStructureCheck(indeks+8,arr)):
        return (arr[0][indeks+7][0] + ' ' + arr[0][indeks+8][0])
    elif(indeks-7>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) 
        and cdStructureCheck(indeks-7,arr)):
        return arr[0][indeks-7][0]
    elif(indeks+7<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) 
        and cdStructureCheck(indeks+7,arr)):
        return arr[0][indeks+7][0]
    elif(indeks-9>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) 
        and cdStructureCheck(indeks-8,arr) and cdStructureCheck(indeks-9,arr)):
        return (arr[0][indeks-9][0] + ' ' + arr[0][indeks-8][0])
    elif(indeks+9<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) 
        and cdStructureCheck(indeks+8,arr) and cdStructureCheck(indeks+9,arr)):
        return (arr[0][indeks+8][0] + ' ' + arr[0][indeks+9][0])
    elif(indeks-8>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) 
        and cdStructureCheck(indeks-8,arr)):
        return arr[0][indeks-8][0]
    elif(indeks+8<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) 
        and cdStructureCheck(indeks+8,arr)):
        return arr[0][indeks+8][0]
    elif(indeks-10>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and cdStructureCheck(indeks-9,arr) and cdStructureCheck(indeks-10,arr)):
        return (arr[0][indeks-10][0] + ' ' + arr[0][indeks-9][0])
    elif(indeks+10<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and cdStructureCheck(indeks+9,arr) and cdStructureCheck(indeks+10,arr)):
        return (arr[0][indeks+9][0] + ' ' + arr[0][indeks+10][0])
    elif(indeks-9>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and cdStructureCheck(indeks-9,arr)):
        return arr[0][indeks-9][0]
    elif(indeks+9<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and cdStructureCheck(indeks+9,arr)):
        return arr[0][indeks+9][0]
    elif(indeks-11>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) 
        and cdStructureCheck(indeks-10,arr) and cdStructureCheck(indeks-11,arr)):
        return (arr[0][indeks-11][0] + ' ' + arr[0][indeks-10][0])
    elif(indeks+11<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) 
        and cdStructureCheck(indeks+10,arr) and cdStructureCheck(indeks+11,arr)):
        return (arr[0][indeks+10][0] + ' ' + arr[0][indeks+11][0])
    elif(indeks-10>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) 
        and cdStructureCheck(indeks-10,arr)):
        return arr[0][indeks-10][0]
    elif(indeks+10<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) 
        and cdStructureCheck(indeks+10,arr)):
        return arr[0][indeks+10][0]
    elif(indeks-12>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) and structureCheck(indeks-10,arr) 
        and cdStructureCheck(indeks-11,arr) and cdStructureCheck(indeks-12,arr)):
        return (arr[0][indeks-12][0] + ' ' + arr[0][indeks-11][0])
    elif(indeks+12<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) and structureCheck(indeks+10,arr) 
        and cdStructureCheck(indeks+11,arr) and cdStructureCheck(indeks+12,arr)):
        return (arr[0][indeks+11][0] + ' ' + arr[0][indeks+12][0])
    elif(indeks-11>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) and structureCheck(indeks-10,arr) 
        and cdStructureCheck(indeks-11,arr)):
        return arr[0][indeks-11][0]
    elif(indeks+11<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) and structureCheck(indeks+10,arr) 
        and cdStructureCheck(indeks+11,arr)):
        return arr[0][indeks+11][0]
    elif(indeks-13>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) and structureCheck(indeks-10,arr) and structureCheck(indeks-11,arr) 
        and cdStructureCheck(indeks-12,arr) and cdStructureCheck(indeks-13,arr)):
        return (arr[0][indeks-13][0] + ' ' + arr[0][indeks-12][0])
    elif(indeks+13<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) and structureCheck(indeks+10,arr) and structureCheck(indeks+11,arr) 
        and cdStructureCheck(indeks+12,arr) and cdStructureCheck(indeks+13,arr)):
        return (arr[0][indeks+12][0] + ' ' + arr[0][indeks+13][0])
    elif(indeks-12>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) and structureCheck(indeks-10,arr) and structureCheck(indeks-11,arr) 
        and cdStructureCheck(indeks-12,arr)):
        return arr[0][indeks-12][0]
    elif(indeks+12<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) and structureCheck(indeks+10,arr) and structureCheck(indeks+11,arr) 
        and cdStructureCheck(indeks+12,arr)):
        return arr[0][indeks+12][0]
    elif(indeks-14>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) and structureCheck(indeks-10,arr) and structureCheck(indeks-11,arr) and structureCheck(indeks-12,arr) 
        and cdStructureCheck(indeks-13,arr) and cdStructureCheck(indeks-14,arr)):
        return (arr[0][indeks-14][0] + ' ' + arr[0][indeks-13][0])
    elif(indeks+14<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) and structureCheck(indeks+10,arr) and structureCheck(indeks+11,arr) and structureCheck(indeks+12,arr) 
        and cdStructureCheck(indeks+13,arr) and cdStructureCheck(indeks+14,arr)):
        return (arr[0][indeks+13][0] + ' ' + arr[0][indeks+14][0])
    elif(indeks-13>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) and structureCheck(indeks-10,arr) and structureCheck(indeks-11,arr) and structureCheck(indeks-12,arr) 
        and cdStructureCheck(indeks-13,arr)):
        return arr[0][indeks-13][0]
    elif(indeks+13<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) and structureCheck(indeks+10,arr) and structureCheck(indeks+11,arr) and structureCheck(indeks+12,arr) 
        and cdStructureCheck(indeks+13,arr)):
        return arr[0][indeks+13][0]
    elif(indeks-15>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) and structureCheck(indeks-10,arr) and structureCheck(indeks-11,arr) and structureCheck(indeks-12,arr) 
        and structureCheck(indeks-13,arr) 
        and cdStructureCheck(indeks-14,arr) and cdStructureCheck(indeks-15,arr)):
        return (arr[0][indeks-15][0] + ' ' + arr[0][indeks-14][0])
    elif(indeks+15<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) and structureCheck(indeks+10,arr) and structureCheck(indeks+11,arr) and structureCheck(indeks+12,arr) 
        and structureCheck(indeks+13,arr) 
        and cdStructureCheck(indeks+14,arr) and cdStructureCheck(indeks+15,arr)):
        return (arr[0][indeks+14][0] + ' ' + arr[0][indeks+15][0])
    elif(indeks-14>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) and structureCheck(indeks-10,arr) and structureCheck(indeks-11,arr) and structureCheck(indeks-12,arr) 
        and structureCheck(indeks-13,arr) 
        and cdStructureCheck(indeks-14,arr)):
        return arr[0][indeks-14][0]
    elif(indeks+14<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) and structureCheck(indeks+10,arr) and structureCheck(indeks+11,arr) and structureCheck(indeks+12,arr) 
        and structureCheck(indeks+13,arr) 
        and cdStructureCheck(indeks+14,arr)):
        return arr[0][indeks+14][0]
    elif(indeks-15>=0 
        and structureCheck(indeks-1,arr) and structureCheck(indeks-2,arr) and structureCheck(indeks-3,arr) and structureCheck(indeks-4,arr) 
        and structureCheck(indeks-5,arr) and structureCheck(indeks-6,arr) and structureCheck(indeks-7,arr) and structureCheck(indeks-8,arr) 
        and structureCheck(indeks-9,arr) and structureCheck(indeks-10,arr) and structureCheck(indeks-11,arr) and structureCheck(indeks-12,arr) 
        and structureCheck(indeks-13,arr) and structureCheck(indeks-14,arr) 
        and cdStructureCheck(indeks-15,arr)):
        return arr[0][indeks-15][0]
    elif(indeks+15<=len(arr[0])-1 
        and structureCheck(indeks+1,arr) and structureCheck(indeks+2,arr) and structureCheck(indeks+3,arr) and structureCheck(indeks+4,arr) 
        and structureCheck(indeks+5,arr) and structureCheck(indeks+6,arr) and structureCheck(indeks+7,arr) and structureCheck(indeks+8,arr) 
        and structureCheck(indeks+9,arr) and structureCheck(indeks+10,arr) and structureCheck(indeks+11,arr) and structureCheck(indeks+12,arr) 
        and structureCheck(indeks+13,arr) and structureCheck(indeks+14,arr) 
        and cdStructureCheck(indeks+15,arr)):
        return arr[0][indeks+15][0]
    else:
        return 1

In [16]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [17]:
def convertTeks(string):
    string = string.lower()
    string = string.split()
    string = [x.replace(',','.') for x in string]
    
    arr = []

    if(len(string)==2 and isfloat(string[0])==True  and string[1]=='juta'):
        hasil="{:.0f}".format(float(string[0])*1000000)
        arr.append(hasil)
    elif(len(string)==2 and isfloat(string[0])==True  and string[1]=='ribu'):
        hasil="{:.0f}".format(float(string[0])*1000)
        arr.append(hasil)
    elif(len(string)==2 and isfloat(string[0])==True  and string[1]=='miliar'):
        hasil="{:.0f}".format(float(string[0])*1000000000)
        arr.append(hasil)
    elif(len(string)==2 and isfloat(string[0])==True  and string[1]=='triliun'):
        hasil="{:.0f}".format(float(string[0])*1000000000000)
        arr.append(hasil)
    else:
        for teks in string:
            # print(teks, type(teks))
            if(teks[-1:]=='%' and (teks[:-1].isdigit() or isfloat(teks[:-1])==True)):
                arr.append(str(teks))
            elif(len(teks)==5 and teks[2]=='.' and isfloat(teks)==True):
                arr.append(str(1))
            elif(len(teks)>=5 and teks[-4]=='.' and isfloat(teks)==True):
                teks=teks.replace('.','')
                arr.append(teks)
            elif(teks=="2010" or teks=="2011" or teks=="2012" or teks=="2013" or teks=="2014" or teks=="2015" or teks=="2016" or teks=="2017" or teks=="2018" or teks=="2019" or teks=="2020" or teks=="2021" or teks=="2022"):
                arr.append(str(1))
            elif(isfloat(teks)==True):
                arr.append(teks)
            elif(teks=="satu" or teks=="seorang" or teks=="seseorang" or teks=="sesosok" or teks=="sang" or teks=="sedikit" or teks=="suatu" or teks=='pertama' or teks=='sebagian' or teks=='paruh'):
                arr.append(str(1))
            elif(teks=="dua" or teks=="kedua" or teks=="keduanya" or teks=="pasangan" or teks=='ke-2'):
                arr.append(str(2))
            elif(teks=="tiga" or teks=="ketiga" or teks=='ke-3'):
                arr.append(str(3))
            elif(teks=="empat" or teks=="keempat" or teks=='ke-4'):
                arr.append(str(4))
            elif(teks=="lima" or teks=="kelima" or teks=='ke-5'):
                arr.append(str(5))
            elif(teks=="enam"  or teks=="keenam" or teks=='ke-6'):
                arr.append(str(6))
            elif(teks=="tujuh"  or teks=="ketujuh" or teks=='ke-7'):
                arr.append(str(7))
            elif(teks=="delapan"  or teks=="kedelapan" or teks=='ke-8'):
                arr.append(str(8))
            elif(teks=="sembilan"  or teks=="kesembilan" or teks=='ke-9'):
                arr.append(str(9))
            elif(teks=="sepuluh"  or teks=="kesepuluh" or teks=="puluhan" or teks=='ke-10'):
                arr.append(str(10))
            elif(teks=="salah" or teks=="sebanyak" or teks=="akibatnya"):
                arr.append('')
            elif(teks=="persen"):
                arr.append('%')
            elif(teks=="belasan"):
                arr.append(str(11))

            #Banyak yang di bawah ini tergantung topik masing2
            elif(teks=="semua"):
                arr.append(str(random.randint(1,5)))
            elif(teks=="sejumlah"):
                arr.append(str(random.randint(6,10)))
            elif(teks=="seluruh"):
                arr.append(str(random.randint(6,15)))
            elif(teks=="beberapa"):
                arr.append(str(random.randint(1,5)))
            elif(teks=="banyak"):
                arr.append(str(random.randint(1,5)))
            elif(teks=="puluh"):
                arr.append(str(0))
            elif(teks=="ratus"):
                arr.append(str(00))
            elif(teks=="ratusan"):
                arr.append(str(100))
            elif(teks=="ribuan"):
                arr.append(str(1000))
            elif(teks=="ribu"):
                arr.append(str(000))
            elif(teks=="jutaan"):
                arr.append(str(1000000))
            elif(teks=="juta"):
                arr.append(str(000000))
            elif(teks=="miliar"):
                arr.append(str(000000000))
            elif(teks=="miliaran"):
                arr.append(str(1000000000))
            elif(teks=="triliun"):
                arr.append(str(000000000000))
            elif(teks=="triliunan"):
                arr.append(str(1000000000000))
            else:
                arr.append(str(1))

    hasil="".join(arr)

    if(hasil[-1:]=='%' and (hasil[:-1].isdigit() or isfloat(hasil[:-1])==True)):
        hasil=float(hasil[:-1])/100
        arr.append(str(hasil))

    return(hasil)

In [18]:
def indeksKe(arr,indikator):
    for x in range(0, len(arr[0])):
        for y in indikator:
            y=y.split()
            if(len(y)==1 and arr[0][x][0]==y[0]):
                return x
            elif(len(y)==2 and len(arr[0])>x and arr[0][x][0]==y[0] and arr[0][x+1][0]==y[1]):
                return x
    return -1

In [19]:
def whatCalculation(index):
    document_result=[]
    document_result_what=[]
    # document_result_provinsi=[]
    
    tfidf_vectorizer = joblib.load('model/QE/vectorizer.pkl')
   
    hasilWhat=[]
    trueWhat=[]

    corpus_teks1=preprocessingCorpus(df_test.iloc[index,-2])
    corpus_teks = (' '.join(corpus_teks1)) 
    tfidf_matrix = tfidf_vectorizer.fit_transform([corpus_teks])

    query_vec_What= tfidf_vectorizer.transform(kandidatFixWhat)
    results_what=cosine_similarity(tfidf_matrix, query_vec_What).reshape((-1))
    # document_result_what.append(corpus_teks)
    for a in kueriFixWithDelimiter_What:
        cariW = re.findall(a,corpus_teks)
        #print(cariW)
        if cariW:
            hasilWhat.append(a)
    
    for x in hasilWhat:
        if((x=='kecelakaan' or x=='tertabrak' or x=='tabrakan' or x=='menabrak' or x=='pesawat jatuh' or x=='kapal' or x=='tenggelam' or x=='terbalik' or x=='kereta api' or x=='pesawat' or x=='terjatuh')and x not in trueWhat):
            trueWhat.append(x)
    
    return(trueWhat,hasilWhat)
   

In [20]:
def getKeparahan(indeks):

    D_Tree = joblib.load("model/D_tree/d_tree.pkl")
    arr =[]
    keparahan_teks =preprocessingKeparahan(df_test.iloc[indeks,-2])
    arr.append(keparahan_teks)
    tag_keparahan = ct.tag_sents(arr)

    #kalau buat yang seperti di bawah ini satu jangan lupa buat if else kayak di bawah
    death_indeks= indeksKe(tag_keparahan,['meninggal','mati','tewas','meregang nyawa','menewaskan','merenggut','jenazah','Jenazah','terlindas','Terlindas','Merenggut','Meninggal','Mati','Tewas','Meregang','Menewaskan']) #kematian #terseret #nyawa #,'jiwa' Merenggut
    injury_indeks= indeksKe(tag_keparahan,['luka-luka','luka','melukai','terluka','kritis','Luka-luka','Luka','Melukai','Terluka','Kritis','perawatan','Perawatan']) #dirawat
    lost_indeks= indeksKe(tag_keparahan,['hilang','Hilang'])
    sink_indeks=indeksKe(tag_keparahan,['tenggelam','tenggelamnya'])
    plane_indeks=indeksKe(tag_keparahan,['pesawat'])
    #kerugian

    if(death_indeks==-1):
        mati = 0
    else:
        mati = struktur1(tag_keparahan,death_indeks)
        if(isinstance(mati, str)):
            mati=convertTeks(mati)

    if(injury_indeks==-1):
        luka = 0
    else:
        luka = struktur1(tag_keparahan,injury_indeks)
        if(isinstance(luka, str)):
            luka=convertTeks(luka)

    if(lost_indeks==-1):
        hilang = 0
    else:
        hilang = struktur1(tag_keparahan,lost_indeks)
        if(isinstance(hilang, str)):
            hilang=convertTeks(hilang)


    if(sink_indeks==-1):
        tenggelam = 0
    else:
        tenggelam = struktur1(tag_keparahan,sink_indeks)
        if(isinstance(tenggelam, str)):
            tenggelam=convertTeks(tenggelam)

    if(plane_indeks==-1):
        pesawat = 0
    else:
        pesawat = 1
    
    keparahan = D_Tree.predict([[mati,luka,hilang,tenggelam,pesawat]])
    if(keparahan[0]==0):
        keparahan="parah"
    elif(keparahan[0]==1):
        keparahan="rendah"
    elif(keparahan[0]==2):
        keparahan="sedang"
    
    # print(mati,luka,hilang,tenggelam,pesawat)
    return(mati,luka,hilang,tenggelam,pesawat,keparahan)


# NER

In [21]:
def compareToDatasetLocation(provinsi,kabupaten,kecamatan):
      trueLocation= checkLocation(provinsi,kabupaten,kecamatan)
      if(len(trueLocation)==3):
            for i in range(0,len(kodePos)):
                  if(kodePos[i][0]==trueLocation[2] and kodePos[i][1]==trueLocation[1] and kodePos[i][2]==trueLocation[0]):
                        return trueLocation
                  
            trueLocation.insert(0,'-')
            trueLocation.insert(1,'-')
            trueLocation.insert(2,'-')
            return trueLocation

      elif(len(trueLocation)==2):
            for i in range(0,len(kodePos)):
                  if(kodePos[i][0]==trueLocation[1] and kodePos[i][1]==trueLocation[0]):
                        trueLocation.insert(0,'-')
                        return trueLocation
                  elif(kodePos[i][1]==trueLocation[1] and kodePos[i][2]==trueLocation[0]):
                        trueLocation.insert(2,'-')
                        return trueLocation
                  elif(kodePos[i][0]==trueLocation[1] and kodePos[i][2]==trueLocation[0]):
                        trueLocation.insert(1,'-')
                        return trueLocation
      
            trueLocation.insert(0,'-')
            trueLocation.insert(1,'-')
            trueLocation.insert(2,'-')
            return trueLocation

      elif(len(trueLocation)==1):
            for i in range(0,len(kodePos)):
                  if(kodePos[i][0]==trueLocation[0] ):
                        trueLocation.insert(0,'-')
                        trueLocation.insert(1,'-')
                        return trueLocation
                  elif(kodePos[i][1]==trueLocation[0] ):
                        trueLocation.insert(0,'-')
                        trueLocation.insert(2,'-')
                        return trueLocation
                  elif(kodePos[i][2]==trueLocation[0] ):
                        trueLocation.insert(1,'-')
                        trueLocation.insert(2,'-')
                        return trueLocation
      
            trueLocation.insert(0,'-')
            trueLocation.insert(1,'-')
            trueLocation.insert(2,'-')
            return trueLocation
            
      else:
            trueLocation.insert(0,'-')
            trueLocation.insert(1,'-')
            trueLocation.insert(2,'-')
            return trueLocation


In [22]:
def checkLocation(provinsi,kabupaten,kecamatan):
    hasil=[]
    if(len(provinsi)>0 and len(kabupaten)>0 and len(kecamatan)>0):
        provinsi_top=provinsi[0]
        kabupaten_top=kabupaten[0]
        kecamatan_top=kecamatan[0]
        if(kecamatan_top[1]<kabupaten_top[1]<provinsi_top[1]):
            if(kabupaten_top[1]-kecamatan_top[2]<=60 and provinsi_top[1]-kabupaten_top[2]<=60):
                hasil.append(kecamatan_top[0])
                hasil.append(kabupaten_top[0])
                hasil.append(provinsi_top[0])
            elif(kabupaten_top[1]-kecamatan_top[2]<=60 and provinsi_top[1]-kabupaten_top[2]>60):
                hasil.append(kecamatan_top[0])
                hasil.append(kabupaten_top[0])
            elif(kabupaten_top[1]-kecamatan_top[2]>60 and provinsi_top[1]-kabupaten_top[2]>60):
                hasil.append((kecamatan_top[0]))
        elif((kecamatan_top[1]<kabupaten_top[1])and(kabupaten_top[1]>provinsi_top[1])and(kecamatan_top[1]<provinsi_top[1])):
            if(provinsi_top[1]-kecamatan_top[2]<=60):
                hasil.append(kecamatan_top[0])
                hasil.append(provinsi_top[0])
            elif(provinsi_top[1]-kecamatan_top[2]>60):
                hasil.append(kecamatan_top[0])
        elif((kecamatan_top[1]>kabupaten_top[1])and(kabupaten_top[1]<provinsi_top[1])and(kecamatan_top[1]>provinsi_top[1])):
            if(provinsi_top[1]-kabupaten_top[2]<=60):
                hasil.append(kabupaten_top[0])
                hasil.append(provinsi_top[0])
            elif(provinsi_top[1]-kabupaten_top[2]>60):
                hasil.append((kabupaten_top[0]))
        elif((kecamatan_top[1]>kabupaten_top[1])and(kabupaten_top[1]<provinsi_top[1])and(kecamatan_top[1]<provinsi_top[1])):
            hasil.append(kabupaten_top[0])
        elif((kecamatan_top[1]<kabupaten_top[1])and(kabupaten_top[1]>provinsi_top[1])and(kecamatan_top[1]>provinsi_top[1])):
            hasil.append(provinsi_top[0])
        elif(kecamatan_top[1]>kabupaten_top[1]>provinsi_top[1]):
            hasil.append(provinsi_top[0])
    elif(len(provinsi)>0 and len(kabupaten)>0 and len(kecamatan)==0):
        provinsi_top=provinsi[0]
        kabupaten_top=kabupaten[0]
        if(kabupaten_top[1]<provinsi_top[1]):
            if(provinsi_top[1]-kabupaten_top[2]<=60):
                hasil.append(kabupaten_top[0])
                hasil.append(provinsi_top[0])
            else:
                hasil.append(kabupaten_top[0])
        elif(kabupaten_top[1]>provinsi_top[1]):
            hasil.append(provinsi_top[0])
    elif(len(provinsi)>0 and len(kabupaten)==0 and len(kecamatan)>0):
        provinsi_top=provinsi[0]
        kecamatan_top=kecamatan[0]
        if(kecamatan_top[1]<provinsi_top[1]):
            if(provinsi_top[1]-kecamatan_top[2]<=60):
                hasil.append(kecamatan_top[0])
                hasil.append(provinsi_top[0])
            else:
                hasil.append(kecamatan_top[0])
        elif(kecamatan_top[1]>provinsi_top[1]):
            hasil.append(provinsi_top[0])
    elif(len(provinsi)==0 and len(kabupaten)>0 and len(kecamatan)>0):
        kabupaten_top=kabupaten[0]
        kecamatan_top=kecamatan[0]
        if(kecamatan_top[1]<kabupaten_top[1]):
            if(kabupaten_top[1]-kecamatan_top[2]<=60):
                hasil.append(kecamatan_top[0])
                hasil.append(kabupaten_top[0])
            else:
                hasil.append(kecamatan_top[0])
        elif(kecamatan_top[1]>kabupaten_top[1]):
            hasil.append(kabupaten_top[0])
    elif(len(provinsi)>0 and len(kabupaten)==0 and len(kecamatan)==0):
        hasil.append(provinsi[0][0])
    elif(len(provinsi)==0 and len(kabupaten)>0 and len(kecamatan)==0):
        hasil.append(kabupaten[0][0])
    elif(len(provinsi)==0 and len(kabupaten)==0 and len(kecamatan)>0):
        hasil.append(kecamatan[0][0])
    elif(len(provinsi)==0 and len(kabupaten)==0 and len(kecamatan)==0):
        return hasil
    
    return hasil



In [23]:
list_provinsi=joblib.load('model/Where/provinsi.pkl')
list_kabupaten=joblib.load('model/Where/kabupaten.pkl')
list_kecamatan=joblib.load('model/Where/kecamatan.pkl')
kodePos = joblib.load('model/Where/listProvKabKec.pkl')

In [24]:
def getNewsTime(dateTime: str):
    month_to_number = {
      'jan': '1',
      'feb': '2',
      'mar': '3',
      'apr': '4',
      'mei': '5',
      'jun': '6',
      'jul': '7',
      'agu': '8',
      'sep': '9',
      'okt': '10',
      'nov': '11',
      'des': '12',
    }

    splitted_timestamp = dateTime.split()
    pre_formatted_time = splitted_timestamp[1] + '/' + month_to_number[splitted_timestamp[2].lower()] + '/' + splitted_timestamp[3]
    
    return pre_formatted_time

In [25]:
def compareDateArray(sortByLength,newsDate):
    year = newsDate.split('/')[2]
    timestampList = []
    
    def convertToTimestapms(date):
        return int(time.mktime(datetime.datetime.strptime(date, "%d/%m/%Y").timetuple()))

    for i, tempTime in enumerate(sortByLength):
        # Find the content inside parentheses
        if(tempTime[-1:]!=')'):
            tempTime=tempTime+')'
            
        currTime = re.search(r'\((.*?)\)', tempTime)
        
        if (currTime != None):
            currTime = currTime.group(1)
            
        currTime = str(currTime)
        
        if (tempTime.lower() == 'kemarin' or ('kemarin' in tempTime.lower())):
            timestampList.append(convertToTimestapms(newsDate) - (24*60*60))
        
        if (currTime != None):
            # Find the year
            splitted = currTime.split('/')

            formattedDate = None
            if (len(splitted) == 3):
                if(len(splitted[2])==2):
                    splitted[2]=year
                    currTime="/".join(splitted)
                year = splitted[2]
                timestampList.append(convertToTimestapms(currTime))
            elif (len(splitted) == 2):
                formattedDate = (splitted[0] + '/' + splitted[1] + '/' + year)
                timestampList.append(convertToTimestapms(formattedDate))
    
    return dt.fromtimestamp(sorted(timestampList)[0]).strftime('%d/%m/%Y')

In [26]:
def toOriginal(teks):
    hasil=''
    if(teks=='sulsel'):
        hasil='sulawesi selatan'
    elif(teks=='sultra'):
        hasil='sulawesi tenggara'
    elif(teks=='sulut'):
        hasil='sulawesi utara'
    elif(teks=='sulteng'):
        hasil='sulawesi tengah'
    elif(teks=='sulbar'):
        hasil='sulawesi barat'
    elif(teks=='kaltim'):
        hasil='kalimantan timur'
    elif(teks=='kaltara'):
        hasil='kalimantan utara'
    elif(teks=='kalteng'):
        hasil='kalimantan tengah'
    elif(teks=='kalsel'):
        hasil='kalimantan selatan'
    elif(teks=='kalbar'):
        hasil='kalimantan barat'
    elif(teks=='jatim'):
        hasil='jawa timur'
    elif(teks=='jateng'):
        hasil='jawa tengah'
    elif(teks=='jabar'):
        hasil='jawa barat'
    elif(teks=='diy'):
        hasil='di yogyakarta'
    elif(teks=='jakbar'):
        hasil='jakarta barat'
    elif(teks=='jaktim'):
        hasil='jakarta timur'
    elif(teks=='jaksel'):
        hasil='jakarta selatan'
    elif(teks=='jakpus'):
        hasil='jakarta pusat'
    elif(teks=='jakut'):
        hasil='jakarta utara'
    elif(teks=='nad'):
        hasil='nanggroe aceh darussalam'
    elif(teks=='sumsel'):
        hasil='sumatera selatan'
    elif(teks=='sumut'):
        hasil='sumatera utara'
    elif(teks=='sumbar'):
        hasil='sumatera barat'
    else:
        hasil=teks
    return hasil

In [27]:
def getNER(indeks,model):

      
      news = preprocessingNER(df_test.iloc[indeks, -2])
      doc = model(news)

      # newsDate = getNewsTime(df_test.iloc[21,1])
      newsDate = getNewsTime(df_test.iloc[indeks,1])

      provinsi = []
      kabupaten = []
      kecamatan = []
      tanggal = []
      orang = []
      visited = []

      tanggal_asli=''

      for ent in doc.ents:
            if(ent.label_== 'LOKASI'):
                  if(ent.text[-1:]=='.'):
                        teks = ent.text[:-1]
                        teks = toOriginal(teks)
                        if ((teks in list_provinsi) and (teks not in visited)):
                              provinsi.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                        elif ((teks in list_kabupaten) and (teks not in visited)):
                              kabupaten.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                        elif ((teks in list_kecamatan) and (teks not in visited)):
                              kecamatan.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                  else:
                        teks = toOriginal(ent.text)
                        if ((teks in list_provinsi) and (teks not in visited)):
                              provinsi.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                        elif ((teks in list_kabupaten) and (teks not in visited)):
                              kabupaten.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                        elif ((teks in list_kecamatan) and (teks not in visited)):
                              kecamatan.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
            elif(ent.label_== 'TANGGAL'):
                  if(ent.text[-1:]=='.'):
                        teks = ent.text[:-1]
                        if (teks not in visited):
                              tanggal.append(teks)
                              visited.append(teks)
                  else:
                        if (ent.text not in visited):
                              tanggal.append(ent.text)
                              visited.append(ent.text)
            elif(ent.label_== 'ORANG'):
                  if(ent.text[-1:]=='.'):
                        teks = ent.text[:-1]
                        if (teks not in visited):
                              orang.append(teks)
                              visited.append(teks)
                  else:
                        if (ent.text not in visited):
                              orang.append(ent.text)
                              visited.append(ent.text)

      visited.clear()

      sortedTimes = sorted(tanggal, key=len, reverse=True)
      # if array is empty return news date
      if(len(sortedTimes) == 0):
            tanggal_asli=newsDate
            # print(newsDate)
      # if array not empty and not contain kemarin
      elif (len(sortedTimes) >= 1):
            tanggal_asli=compareDateArray(sortedTimes,newsDate)
            # print(tanggal_asli)
            # print(compareDateArray(sortedTimes,newsDate))

      lokasi_asli = compareToDatasetLocation(provinsi,kabupaten,kecamatan)

      return (orang,tanggal_asli,lokasi_asli[2],lokasi_asli[1],lokasi_asli[0])

In [31]:
def cekhasil():
    document_result=[]

    link_to_model = "model/NER_Kec_Ben_Krim_Kese_Eko/output/model-last"
    loaded_model = load(link_to_model)
    
    
    for i in range(0, len(df_test)):
        print(i)
        
        trueWhat,hasilWhat = whatCalculation(i)

        if(len(trueWhat)>0):
            truePos=1
        else:
            truePos=0

        mati,luka,hilang,tenggelam,pesawat,keparahan = getKeparahan(i)

        orang,tanggal_asli,provinsi,kabupaten,kecamatan=getNER(i,loaded_model)

        data = [i,df_test.iloc[i,0],df_test.iloc[i,1],df_test.iloc[i,2],trueWhat,truePos, hasilWhat,orang,tanggal_asli,provinsi,kabupaten,kecamatan,mati,luka,hilang,tenggelam,pesawat,keparahan]
        document_result.append(data)
    
    writer = pd.DataFrame(document_result, columns=['No Document','title','Date','Description', 'What','TruePos', 'hasilWhat','orang','tanggal_asli','provinsi','kabupaten','kecamatan',"mati",'luka','hilang','tenggelam','pesawat','keparahan'])
    writer.to_csv('result/QE_Stat_testing_result5.csv', index=False, sep=',')



In [32]:
cekhasil()

0


d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


1
2


d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


3
4


...

d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


95
96


d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


97
98
99


d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
